<a href="https://colab.research.google.com/github/shibukawar/stein-importance-sampling-qa/blob/feature%2Fstein-importance-sampling/notebook/example_stein_importance_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiments on free energy and magnetization

In [ ]:
import os
os.chdir('../')

In [ ]:
import os

import numpy as np

from stein.energy import create_hamiltonian
from stein.util import pdump, read_from_file
from collections import defaultdict

In [ ]:
PROBLEM = "GSD_8"

In [ ]:
PROBLEM_DIR_PATH = f"./problems/{PROBLEM}/"

J, h = read_from_file(PROBLEM_DIR_PATH)
hamiltonian = create_hamiltonian(J, h)

# Define a target distribution

In [ ]:
def boltzmann_factor(x, hamiltonian, beta):
    s = hamiltonian(x)
    return np.exp(-beta * s)

In [ ]:
def mh(dim=16, beta=0.1, N=10000):
    accepted_samples = []
    x = 2 * np.random.randint(2, size=dim) - 1
    for _ in range(N):
        y = 2 * np.random.randint(2, size=dim) - 1
        alpha = boltzmann_factor(y, hamiltonian=hamiltonian, beta=beta) / boltzmann_factor(x, hamiltonian=hamiltonian, beta=beta)
        a = np.random.uniform(0, 1)
        if a < alpha:
            accepted_samples.append(y)
            x = y
        else:
            accepted_samples.append(x)
    return accepted_samples

dim = 16
N = 10000
num = 2000
ene_result_2000, u4_result_2000, sus_result_2000 = [], [], []
samples_dict = defaultdict(list)
for beta in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    # print(beta)
    free_energy_list_2000, u4_list_2000, susceptibility_list_2000 = [], [], []
    for _ in range(5):
        accepted_samples = mh(dim=dim, beta=beta, N=N)
        samples_dict[beta].append(accepted_samples)
        n = len(accepted_samples)
        free_energy, susceptibility = 0, 0
        m2, m4 = 0, 0
        num = 2000
        for x in accepted_samples[n-num:]:
            m = sum(x)
            free_energy += hamiltonian(x)
            m2 += m ** 2
            m4 += m ** 4
            susceptibility += m ** 2 * beta
        free_energy /= num
        susceptibility /= num
        m2 /= num
        m4 /= num
        u4 = 1 - m4 / (3 * m2**2)
        free_energy_list_2000.append(free_energy)
        u4_list_2000.append(u4)
        susceptibility_list_2000.append(susceptibility)
    ene_result_2000.append(free_energy_list_2000)
    u4_result_2000.append(u4_list_2000)
    sus_result_2000.append(susceptibility_list_2000)
ene_result_2000 = np.array(ene_result_2000)
u4_result_2000 = np.array(u4_result_2000)
sus_result_2000 = np.array(sus_result_2000)

In [ ]:
result_dict = {}
result_dict['energy'] = ene_result_2000
result_dict['u4'] = u4_result_2000
result_dict['sus'] = sus_result_2000
path = os.path.join(PROBLEM_DIR_PATH, "mcmc_2000.pickle.gz2")
pdump(path, result_dict)